In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [2]:
%%capture
!pip install "git+https://github.com/huggingface/transformers.git"

In [1]:
from datasets import load_dataset
import torch

In [2]:
dataset=load_dataset("imdb", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [4]:
dataset[0].get("text"), dataset[0].get("label")

('I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [ ]:
%%capture
!pip install xformers

In [5]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [6]:
max_seq_length=2048

In [7]:
model, tokenizer=FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2024.10.7: Fast Mistral patching. Transformers = 4.47.0.dev0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [8]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((40

In [9]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/mistral-7b-bnb-4bit', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
print(torch.cuda.is_available)

<function is_available at 0x7f4a9813b880>


In [17]:
%%capture
!pip install --upgrade pip
!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

In [19]:
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 10,
    max_steps = 60,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    output_dir = "outputs",
    optim = "adamw_8bit",
    seed = 3407,
    )

In [21]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = args,
    tokenizer = tokenizer,

)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [22]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,0.609100
2,0.586800
3,0.597300
4,0.613700
5,0.647100
6,0.681400
7,0.576800
8,0.582500
9,0.552300
10,0.616000


TrainOutput(global_step=60, training_loss=0.5994496057430904, metrics={'train_runtime': 770.9392, 'train_samples_per_second': 0.623, 'train_steps_per_second': 0.078, 'total_flos': 9642624560529408.0, 'train_loss': 0.5994496057430904, 'epoch': 0.0192})

In [52]:
review="Stupid Movie, How can someone even think of producing such a shitty movie. Eww - Can you tell me if its a positive or a negative review"

In [53]:
input=tokenizer([review], return_tensors="pt", padding=True).to("cuda")

In [54]:
input

{'input_ids': tensor([[    1,   662,  8064, 23208, 28725,  1602,   541,  2493,  1019,  1073,
           302, 15345,  1259,   264, 23933,  5994, 28723,   413,  1566,   387,
          2418,   368,  1912,   528,   513,   871,   264,  5278,   442,   264,
          7087,  4058]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [55]:
model=FastLanguageModel.for_inference(model)

In [56]:
output = model.generate(**input, max_new_tokens=128, use_cache=True )

In [57]:
output

tensor([[    1,   662,  8064, 23208, 28725,  1602,   541,  2493,  1019,  1073,
           302, 15345,  1259,   264, 23933,  5994, 28723,   413,  1566,   387,
          2418,   368,  1912,   528,   513,   871,   264,  5278,   442,   264,
          7087,  4058, 28804,   315, 28742, 28719,   459,  1864,   513,   315,
          1023,  3054,   378,   442,   459, 28723,   315, 28742, 28719,   459,
          1864,   513,   315,  1023,  3054,   378,   442,   459, 28723,   315,
         28742, 28719,   459,  1864,   513,   315,  1023,  3054,   378,   442,
           459, 28723,   315, 28742, 28719,   459,  1864,   513,   315,  1023,
          3054,   378,   442,   459, 28723,   315, 28742, 28719,   459,  1864,
           513,   315,  1023,  3054,   378,   442,   459, 28723,   315, 28742,
         28719,   459,  1864,   513,   315,  1023,  3054,   378,   442,   459,
         28723,   315, 28742, 28719,   459,  1864,   513,   315,  1023,  3054,
           378,   442,   459, 28723,   315, 28742, 2

In [58]:
tokenizer.batch_decode(output)

["<s> Stupid Movie, How can someone even think of producing such a shitty movie. Eww - Can you tell me if its a positive or a negative review? I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it or not. I'm not sure if I should watch it"]

In [31]:
model.save_pretrained("LORA_Model")